In [65]:
import pandas as pd
import plotly.express as px

In [66]:
layout_options = {
    'paper_bgcolor':"#383838",
    'plot_bgcolor':'#383838',
    'title_font': dict(color='white'),
    'legend_font': dict(color='white'),
    'yaxis':dict(color="white"),
    'xaxis':dict(color="white")
    }

In [67]:
goals = pd.read_csv("data/fjelstul/goals.csv")
players = pd.read_csv("data/fjelstul/players.csv")

معرفة عمر اللاعب عند تسجيل الهدف

In [68]:
(players['birth_date'] == 'not available').sum()

77

In [69]:
players = players.dropna()
players = players.drop(index=players[players['birth_date'] == 'not available'].index)

In [70]:
goals["tournament_year"] = goals.tournament_id.str.split("-").str.get(1)
players["birth_year"] = players.birth_date.str.split("-").str.get(0)
players_goals = pd.merge(
    goals,
    players,
    on="player_id",
    how="inner"
)
players_goals["player_age"] = players_goals["tournament_year"].astype(int) -  players_goals["birth_year"].astype(int)

In [71]:
players_goals["player_age"].describe()

count    2719.000000
mean       26.652814
std         3.465496
min        18.000000
25%        24.000000
50%        26.000000
75%        29.000000
max        42.000000
Name: player_age, dtype: float64

In [72]:
goals_age = players_goals[["player_age", "goal_id"]].groupby("player_age").count()

In [73]:
fig = px.bar(goals_age, y="goal_id")
fig.update_layout(**layout_options)
fig.show()

من الملاحظ أن اللاعب في بداية عمره تكون فرصة تسجيله للاهداف قليلة وكذلك بالنسبة لمن هو فوق عمر 30 سنة، والعمر الأكثر فرصة لتسديد الأهداف هو 25

-----------------------------

حالات التبديل في البطولة

In [75]:
substitutions = pd.read_csv("data/fjelstul/substitutions.csv")
substitutions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7637 entries, 0 to 7636
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   key_id             7637 non-null   int64 
 1   substitution_id    7637 non-null   object
 2   tournament_id      7637 non-null   object
 3   tournament_name    7637 non-null   object
 4   match_id           7637 non-null   object
 5   match_name         7637 non-null   object
 6   match_date         7637 non-null   object
 7   stage_name         7637 non-null   object
 8   group_name         7637 non-null   object
 9   team_id            7637 non-null   object
 10  team_name          7637 non-null   object
 11  team_code          7637 non-null   object
 12  home_team          7637 non-null   int64 
 13  away_team          7637 non-null   int64 
 14  player_id          7637 non-null   object
 15  family_name        7637 non-null   object
 16  given_name         7637 non-null   object


In [80]:
team_max_substitute = substitutions[["tournament_id", "team_id", "substitution_id"]].groupby(
    ["tournament_id", "team_id"]
).count().reset_index().groupby("tournament_id").max().reset_index()

In [79]:
fig = px.bar(team_max_substitute, x="tournament_id", y="substitution_id")
fig.update_layout(**layout_options)
fig.show()

-----------------------------------

احتمال فوز الفريق اذا كان الحكم من نفس الدولة